# Function Calling with Claude 3 on Amazon Bedrock

Function calling with Claude on Amazon Bedrock is available with legacy tool use structure. An updated version of tool use with Claude will be available on Bedrock soon. Reference [tool-use for more information on this updated feature set](https://docs.anthropic.com/claude/docs/tool-use).

With function calling, we can provide Claude with descriptions of tools and functions it can use, Claude is able to intelligently decide based on user query when and how to use those tools to help answer questions and complete tasks. In its response, Claude will suggest the tools to use to complete a task(s) and the paramters that should be passed when calling those functions. The function and its parameters are typically output by Claude in a well-defined format so that the client can parse and run the associated functions. Reference [legacy function calling with Claude](https://docs.anthropic.com/claude/docs/legacy-tool-use#example-legacy-tool-use-prompt).

Behind the scenes, this is a multi-step process:

1. The function definitions and user question are both passed to Claude in a single prompt
2. Claude not only needs the tools & their descriptions in order to successfully decide whether to use the tools, but likely also accompanying examples of situations in which such tools ought to be used, depending on the complexity of the use case and tools.
3. Claude assesses the user's question and decides which function(s) to call and with what arguments
4. Claude constructs a properly formatted function call
5. The function call is intercepted via client code with a clear stop_sequence, and the actual function is executed on the client side
6. The function result is passed back to Claude
7. Claude uses the function result to formulate its final response to the user

Let's Look at some examples and use cases.

## Setup

In [ ]:
%pip install --upgrade pip
%pip install boto3>=1.33.2 --force-reinstall --quiet
%pip install botocore>=1.33.2 --force-reinstall --quiet
%pip install json --quiet
%pip install beautifulsoup4
%pip install lxml

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [2]:
from bs4 import BeautifulSoup 
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime',region_name='us-east-1')


### Define some helper functions

In [60]:
# Generate response and suggested function call using the invoke method from Bedrock runtime

def generate_function_call(bedrock_runtime, model_id, messages, max_tokens=512,top_p=1,temp=0,system=''):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p,
            "system": system,
            "stop_sequences":["</function_calls>"]
        }  
    )  
    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())

    return response_body

In [47]:
# Toy function for outputting ticker symbol from company name

def get_ticker_symbol(company_name):
    
    assert company_name.lower() == "general motors", 'TickerNotFound'
    
    return 'GM'

In [5]:
# Format the functions results for input back in to Claude

def func_results(tool_name,tool_return):
   return f'''
<function_results>
<result>
<tool_name>{tool_name}</tool_name>
<stdout>
{tool_return}
</stdout>
</result>
</function_results>'''

In [6]:
# Parses the output of Claude to extract the suggested function call and parameters

def parse_output(response):
    soup=BeautifulSoup(response['content'][0]['text'].replace('\n',''),"lxml")
    tool_name=soup.tool_name.string
    parameter_name=soup.parameters.contents[0].name
    parameter_value=soup.parameters.contents[0].string
    return (tool_name,{parameter_name:parameter_value})
    

## Notebook Walkthrough

In this notebook, we will take a look at function calling with Claude. We will use the  __MessagesAPI__ format and utilize the __invoke_model__ function from __bedrock-runtime__ within our helper functions to generate a response from Claude. We will also utilize some simple orchestration code to form a loop that will allow Claude to take in the responses from the function call suggested to process and make further suggestions on the next function to call.

## Function Calling Prompt

The prompt that we provide to claude when implementating function calling include several important components:

1. An instruction that describes the intent and sets the context for function calling.
2. A detailed description of the tool(s) and expected parameters that Claude can suggest the use of.
3. An example of the structure of the function call so that it can be parsed by the client code and ran.
4. A directive to form a thought process before deciding on a function to call.
5. The user query itself

Below we will pass our initial prompt with some tools that can be utilized in this toy use-case involving finding the ticker associated with a company. This will involve a function that we will describe to Claude using a tool schema format so it knows what functionlity it has at its disposal. 


In [ ]:
message_list = [
    {
        "role": 'user',
        "content": [ {"type": "text", "text":
       '''In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this:

<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
<tool_description>
<tool_name>get_ticker_symbol</tool_name>
<description>Gets the stock ticker symbol for a company searched by name. Returns str: The ticker symbol for the company stock. Raises TickerNotFound: if no matching ticker symbol is found.</description>
<parameters>
<parameter>
<name>company_name</name>
<type>string</type>
<description>The name of the company.</description>
</parameter>
</parameters>
</tool_description>
</tools>

Come up with a step by step plan for what steps should be taken, what functions should be called and in 
what order. Place your thinking between <rationale> tags. Only create this rationale 1 time before 
creating any other outputs.

You will take in any outputs from called functions which will be in <function_results> tags and use 
them to further suggests next steps and actions to take.


User Query: What is the ticker symbol of General Motors?'''}
        ]
    }
]

response=generate_function_call(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=message_list,max_tokens=512,temp=0.5,top_p=0.9)
print(response['content'][0]['text'])
message_list.append({
        "role": 'assistant',
        "content": [
            {"type": "text", "text": response['content'][0]['text']}
        ]})

As we can see, because we told Claude to end its suggestion for a function call with a </function_calls> tag and make that a stop sequence, we can now parse the output at the <function_calls> tags and extract the function name we should call and the parameters we should use during our call.

__Note__: We designated the stop sequence to be __<"/function_calls">__ so that Claude stops outputting additional text when it gets to that sequence so that we can intercept the suggested function call and read it off. See the above __generate_message__ function definition for details.

In [55]:
tool_name,param=parse_output(response)

In [56]:
try:
    tool_return=eval(tool_name)(**param)
except AssertionError as e:
    tool_return=e

In [ ]:
# We need to place the function results in an input message to Claude with the following structure:

'''<function_results>
   <result>
   <tool_name>get_ticker_symbol</tool_name>
   <stdout>
   function_results
   </stdout>
   </result>
   </function_results>'''

We will call our function results formatter to do this for us

In [57]:
results=func_results(tool_name,tool_return)

Next we will append these results as another message to our conversation history and pass to Claude

In [62]:
message_list.append({
        "role": 'user',
        "content": [
            {"type": "text", "text":f"""This is the final answer to the user question using the function 
            results. Do not output the name of the functions and tools used to get the answer {results}"""}
        ]})

In [ ]:
response=generate_function_call(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=message_list,max_tokens=512,temp=0.5,top_p=0.9)
print(response['content'][0]['text'])

Great! We can see that Claude summarizes the results of the function given the context of the conversation history and answers our original question.

## Next Steps

Now that we have seen how to use tools with Claude using the legacy functionality. You can try adding more complexity, including more tools for Claude to use, orchestration loops, a detailed conversation history, and more complicated questions to ask Claude that uses those tools in different ways.
